## Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' 
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## Dataset Processing 

### Read in the original dataset 

In [2]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,0:2]

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,0:2]

TimeoutError: [Errno 60] Operation timed out

### CSI Processing: Take Modulus of complex matrices

In [ ]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [ ]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)
print(train_label.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
torch.Size([15000, 2])


###  CSI Processing: Normalize to [0,1]

In [ ]:
#normalize to [0,1]
train_CSI_modulus = train_CSI_modulus/torch.max(train_CSI_modulus)
valid_CSI_modulus = valid_CSI_modulus/torch.max(valid_CSI_modulus)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


### CSI to Linear Regression

In [ ]:
import time
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


rstate = 23

ridge = Ridge(alpha=0.1, random_state=rstate)
ridge.fit(train_CSI_modulus.reshape(train_CSI_modulus.shape[0], -1), train_label)

start_time = time.time()
train_pred = ridge.predict(train_CSI_modulus.reshape(train_CSI_modulus.shape[0], -1))
train_mse = mean_squared_error(train_label, train_pred)
end_time = time.time()

start_valid_time = time.time()
valid_pred = ridge.predict(valid_CSI_modulus.reshape(valid_CSI_modulus.shape[0], -1))
valid_mse = mean_squared_error(valid_label, valid_pred)
end_valid_time = time.time()

print('train mse: ', train_mse)
print('valid mse: ', valid_mse)

#root mean square error
print('train rmse: ', math.sqrt(train_mse))
print('valid rmse: ', math.sqrt(valid_mse))

print('Training time taken: {:.2f}s'.format(end_time - start_time))
print('Validation time taken: {:.2f}s'.format(end_valid_time - start_valid_time))

train mse:  10.803894014181738
valid mse:  27.805356569705488
train rmse:  3.286927747027874
valid rmse:  5.273078471794772
Training time taken: 0.56s
Validation time taken: 0.17s


### CSI to ANN

In [ ]:
import time
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(26112, 128)
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        #print('shape 0 ' + str(x.shape))
        x = self.pool(self.relu(self.conv1(x)))
        #print('shape 1 ' + str(x.shape))
        x = self.pool(self.relu(self.conv2(x)))
        #print('shape 2 ' + str(x.shape))
        x = x.view(x.size(0), -1)
        #print('shape 3 ' + str(x.shape))
        x = self.relu(self.fc1(x))
        #print('shape 4 ' + str(x.shape))
        x = self.fc2(x)
        #print('shape 5 ' + str(x.shape))
        return x

# Define the neural network architecture
model = ANN()


# create tensor datasets
train_dataset = TensorDataset(train_CSI_modulus, train_label)
valid_dataset = TensorDataset(valid_CSI_modulus, valid_label)

print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

# create dataloaders
batch_size = 32
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
num_epochs = 10
train_start_time = time.time()
for epoch in range(num_epochs):
    running_loss = 0.0
    epoch_start_time = time.time()
    for i, data in enumerate(train_dl):
        #if i%150==0 or i==len(train_dl)-1:
        #    print('shape set ' + str(i) + ' of ' + str(len(train_dl)) + ' ' + str(inputs.shape))
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time
    print('Epoch [{}/{}], loss: {:.4f}, time: {:.2f}s'.format(epoch+1, num_epochs, running_loss/len(train_dl), epoch_time))
train_end_time = time.time()
train_time = train_end_time - train_start_time
print('Finished training. Total time: {:.2f}s'.format(train_time))

# Evaluate the neural network on the validation set
start_time = time.time()
with torch.no_grad():
    valid_outputs = model(valid_CSI_modulus)
    valid_loss = criterion(valid_outputs, valid_label)
end_time = time.time()
total_time = end_time - start_time
print('Validation loss: {:.4f}, total time: {:.2f}s'.format(valid_loss.item(), total_time))
print('Validation rmse: {:.4f}'.format(math.sqrt(valid_loss.item())))


torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
shape set 0 of 469 torch.Size([24, 1, 4, 1632])


KeyboardInterrupt: 